In [ ]:
! pip install datasets
! pip install pycaret


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 5.2 MB/s eta 0:00:00
 

In [ ]:
from datasets import load_dataset
import gensim.downloader as api
import numpy as np

# Check if GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the GloVe model
glove_model = api.load("glove-wiki-gigaword-100")  # Use the GloVe embeddings (100-dimensional)

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
# def get_sentence_vector(sentence):
#     words = sentence.split()
#     word_vectors = [glove_model[word] for word in words if word in glove_model]
#     return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(glove_model.vector_size)
dataset = load_dataset("google-research-datasets/paws","labeled_final")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [ ]:
np.hstack([[1,2],[2,3]]).tolist()

[1, 2, 2, 3]

In [ ]:
# Ensure get_sentence_vector outputs lists of float32
def get_sentence_vector(sentence):
    words = sentence.split()
    word_vectors = [glove_model[word] for word in words if word in glove_model]
    avg_vector = np.mean(word_vectors, axis=0) if word_vectors else np.zeros(glove_model.vector_size)
    return avg_vector.astype(np.float32).tolist()  # Convert to list of float32

# Apply mapping with updated function
for split in ['train', 'validation', 'test']:
    dataset[split] = dataset[split].map(
        lambda x: {
            'ph1Vec': get_sentence_vector(x['sentence1']),
            'ph2Vec': get_sentence_vector(x['sentence2'])
        }
    )

# Apply mapping with updated function
for split in ['train', 'validation', 'test']:
    dataset[split] = dataset[split].map(
        lambda x: {
            'Vec': ((np.array(x['ph1Vec']) + np.array(x['ph2Vec'])) / 2).tolist()

        }
    )


# Apply mapping with updated function
for split in ['train', 'validation', 'test']:
    dataset[split] = dataset[split].map(
        lambda x: {


            'hVec': np.hstack([x['ph1Vec'],x['ph2Vec']]).tolist()

        }
    )

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
dataset

In [ ]:
# ! pip install "cuml-cu12==23.06" -f https://rapids.ai/wheels/stable


# ! pip3 install auto-sklearn

In [ ]:
X_train = np.array(dataset['train']['hVec'])
y_train = np.array(dataset['train']['label'])
X_val = np.array(dataset['validation']['hVec'])
y_val = np.array(dataset['validation']['label'])

In [ ]:
import pandas as pd
train_df = pd.DataFrame(X_train)
train_df['label'] = y_train  # Add the target variable as a column


In [ ]:
# import pycaret classification and init setup
from pycaret.classification import *
s = setup(train_df, target = 'label', session_id = 123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Original data shape,"(49401, 201)"
4,Transformed data shape,"(49401, 201)"
5,Transformed train set shape,"(34580, 201)"
6,Transformed test set shape,"(14821, 201)"
7,Numeric features,200
8,Preprocess,True
9,Imputation type,simple


In [ ]:
# import ClassificationExperiment and init the class
from pycaret.classification import ClassificationExperiment
exp = ClassificationExperiment()

In [ ]:
exp.setup(train_df, target = 'label', session_id = 123)



,Description,Value
0,Session id,123
1,Target,label
2,Target type,Binary
3,Original data shape,"(49401, 201)"
4,Transformed data shape,"(49401, 201)"
5,Transformed train set shape,"(34580, 201)"
6,Transformed test set shape,"(14821, 201)"
7,Numeric features,200
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best = compare_models()


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:25:38
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.5749,0.5302,0.1414,0.5776,0.2271,0.0645,0.0951,69.2230
lr,Logistic Regression,0.5671,0.5423,0.1926,0.5281,0.2821,0.0603,0.0760,1.0240
ridge,Ridge Classifier,0.5660,0.5408,0.1961,0.5241,0.2853,0.0588,0.0736,0.1330
lda,Linear Discriminant Analysis,0.5652,0.5403,0.1995,0.5209,0.2884,0.0579,0.0718,0.2730
ada,Ada Boost Classifier,0.5622,0.5292,0.2185,0.5114,0.3059,0.0562,0.0673,12.5730
nb,Naive Bayes,0.5424,0.5445,0.4384,0.4805,0.4584,0.0637,0.0639,0.1380
svm,SVM - Linear Kernel,0.5413,0.5363,0.2893,0.5394,0.2746,0.0312,0.0488,0.5610
qda,Quadratic Discriminant Analysis,0.5181,0.5066,0.4287,0.4523,0.4400,0.0176,0.0177,0.3490
knn,K Neighbors Classifier,0.5038,0.4636,0.2951,0.4137,0.3444,-0.0369,-0.0383,1.2720
xgboost,Extreme Gradient Boosting,0.4862,0.4403,0.3074,0.3955,0.3459,-0.0663,-0.0677,3.5100


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

array([0, 1], dtype=int8)